In [2]:
!pip install pymongo pandas

Defaulting to user installation because normal site-packages is not writeable
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 3.2 MB/s eta 0:00:00m eta 0:00:010:01:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 1.6 MB/s eta 0:00:00m eta 0:00:010:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 331.1/331.1 KB 966.3 kB/s eta 0:00:00 MB/s eta 0:00:01:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.8/16.8 MB 1.4 MB/s eta 0:00:00m eta 0:00:010:00:01m
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  NOTE: The current PATH contains path(s) starting with `~`, which may not be expanded by all applications.


In [11]:
# 1. Menghubungkan ke Database MongoDB
# Tugas 1: Pastikan MongoDB telah terpasang dan berjalan. Hubungkan ke MongoDB lokal.

from pymongo import MongoClient

# Inisialisasi client dan koneksi ke database
client = MongoClient('mongodb://localhost:27017/')
db = client['company_db']
collection = db['employees']

# Contoh untuk memastikan koneksi
print('Connected to MongoDB')

Connected to MongoDB


In [30]:
# 2. Operasi CRUD Dasar
# Tugas 2: Insert, Update, dan Delete data pada koleksi employees.
import random

# Daftar departemen
departments = ['Finance', 'HR', 'IT', 'Marketing', 'Sales', 'Operations']

# Insert 100 data karyawan
for i in range(1, 101):
    employee_data = {
        'employee_id': i,
        'name': f'Employee_{i}',
        'department': random.choice(departments),
        'age': random.randint(21, 55),
        'salary': random.randint(3000, 12000)
    }
    collection.insert_one(employee_data)

print("✅ 100 data karyawan berhasil ditambahkan ke koleksi 'employees'")

# Menampilkan 10 data pertama untuk memastikan data sudah masuk
print("\n📋 Data yang berhasil ditambahkan (10 data pertama):")
for emp in collection.find().limit(10):
    print(emp)

# Update Data
result = collection.update_many(
    {'department': 'IT'},  # kondisi
    {'$mul': {'salary': 1.1}}  # operasi: kalikan salary 1.1 (naik 10%)
)
print(f"\n✅ {result.modified_count} data karyawan departemen IT berhasil diperbarui.")

# Delete Data
result = collection.delete_many({'age': {'$gt': 50}})
print(f"🗑️ {result.deleted_count} data karyawan dengan usia > 50 tahun berhasil dihapus.")

print("Jumlah data tersisa:", collection.count_documents({}))

✅ 100 data karyawan berhasil ditambahkan ke koleksi 'employees'

📋 Data yang berhasil ditambahkan (10 data pertama):
{'_id': ObjectId('690439e58d6b2495677813c2'), 'employee_id': 1, 'name': 'Employee_1', 'department': 'Sales', 'age': 34, 'salary': 5980}
{'_id': ObjectId('690439e58d6b2495677813c3'), 'employee_id': 2, 'name': 'Employee_2', 'department': 'Finance', 'age': 54, 'salary': 8100}
{'_id': ObjectId('690439e58d6b2495677813c4'), 'employee_id': 3, 'name': 'Employee_3', 'department': 'Marketing', 'age': 24, 'salary': 7808}
{'_id': ObjectId('690439e58d6b2495677813c5'), 'employee_id': 4, 'name': 'Employee_4', 'department': 'IT', 'age': 35, 'salary': 7945}
{'_id': ObjectId('690439e58d6b2495677813c6'), 'employee_id': 5, 'name': 'Employee_5', 'department': 'Operations', 'age': 26, 'salary': 6629}
{'_id': ObjectId('690439e58d6b2495677813c7'), 'employee_id': 6, 'name': 'Employee_6', 'department': 'Sales', 'age': 44, 'salary': 3449}
{'_id': ObjectId('690439e58d6b2495677813c8'), 'employee_id'

In [31]:
# Contoh Query Data
print("📋Data Karyawan dari Koleksi 'employees'📋\n")
for employee in collection.find():
    print(employee)

📋Data Karyawan dari Koleksi 'employees'📋

{'_id': ObjectId('690439e58d6b2495677813c2'), 'employee_id': 1, 'name': 'Employee_1', 'department': 'Sales', 'age': 34, 'salary': 5980}
{'_id': ObjectId('690439e58d6b2495677813c4'), 'employee_id': 3, 'name': 'Employee_3', 'department': 'Marketing', 'age': 24, 'salary': 7808}
{'_id': ObjectId('690439e58d6b2495677813c5'), 'employee_id': 4, 'name': 'Employee_4', 'department': 'IT', 'age': 35, 'salary': 8739.5}
{'_id': ObjectId('690439e58d6b2495677813c6'), 'employee_id': 5, 'name': 'Employee_5', 'department': 'Operations', 'age': 26, 'salary': 6629}
{'_id': ObjectId('690439e58d6b2495677813c7'), 'employee_id': 6, 'name': 'Employee_6', 'department': 'Sales', 'age': 44, 'salary': 3449}
{'_id': ObjectId('690439e58d6b2495677813c9'), 'employee_id': 8, 'name': 'Employee_8', 'department': 'IT', 'age': 30, 'salary': 10913.1}
{'_id': ObjectId('690439e58d6b2495677813ca'), 'employee_id': 9, 'name': 'Employee_9', 'department': 'Sales', 'age': 23, 'salary': 3073

In [32]:
# 3. Query Lebih Kompleks Menggunakan Aggregation
# Tugas 3: Terapkan aggregation untuk menghitung rata-rata gaji per departemen.

# Query Aggregation untuk mencari rata-rata gaji per departemen
pipeline = [
    {'$group': {'_id': '$department', 'average_salary': {'$avg': '$salary'}}}
]

print("📊 Rata-rata Gaji per Departemen:\n")
for result in collection.aggregate(pipeline):
    print(result)

📊 Rata-rata Gaji per Departemen:

{'_id': 'HR', 'average_salary': 7553.4}
{'_id': 'Sales', 'average_salary': 5792.363636363636}
{'_id': 'Operations', 'average_salary': 8471.8125}
{'_id': 'Marketing', 'average_salary': 5800.25}
{'_id': 'IT', 'average_salary': 8993.966666666667}
{'_id': 'Finance', 'average_salary': 8413.0}


In [33]:
# 5. TUGAS
# TUGAS 1: Aggregation Pipeline untuk mencari 5 karyawan dengan gaji tertinggi di setiap departemen

pipeline_top5 = [
    # Urutkan semua data berdasarkan 'department' (ascending) dan 'salary' (descending)
    {'$sort': {'department': 1, 'salary': -1}},

    # Kelompokkan berdasarkan departemen
    {'$group': {
        '_id': '$department',
        'top_employees': {
            '$push': {  # Simpan semua data karyawan dalam daftar (array)
                'name': '$name',
                'salary': '$salary',
                'age': '$age',
                'employee_id': '$employee_id'
            }
        }
    }},

    # Ambil hanya 5 data karyawan teratas (gaji tertinggi) dalam setiap departemen
    {'$project': {
        '_id': 1,
        'top_5_employees': {'$slice': ['$top_employees', 5]}  # Ambil 5 elemen pertama dari array
    }}
]

results = collection.aggregate(pipeline_top5)

# Tampilkan hasil 
print("🏆 5 Karyawan dengan Gaji Tertinggi per Departemen\n")
for dept in results:
    print(f"Departemen: {dept['_id']}")
    print("Top 5 Karyawan:")
    for emp in dept['top_5_employees']:
        print(f"{emp['name']} | Gaji: ${emp['salary']} | Usia: {emp['age']} | ID: {emp['employee_id']}")
    print()

🏆 5 Karyawan dengan Gaji Tertinggi per Departemen

Departemen: IT
Top 5 Karyawan:
Employee_24 | Gaji: $12162.7 | Usia: 28 | ID: 24
Employee_35 | Gaji: $11540.1 | Usia: 37 | ID: 35
Employee_32 | Gaji: $10976.900000000001 | Usia: 38 | ID: 32
Employee_8 | Gaji: $10913.1 | Usia: 30 | ID: 8
Employee_90 | Gaji: $10392.800000000001 | Usia: 24 | ID: 90

Departemen: Sales
Top 5 Karyawan:
Employee_34 | Gaji: $8941 | Usia: 33 | ID: 34
Employee_27 | Gaji: $7854 | Usia: 34 | ID: 27
Employee_51 | Gaji: $7401 | Usia: 43 | ID: 51
Employee_67 | Gaji: $7326 | Usia: 41 | ID: 67
Employee_16 | Gaji: $6419 | Usia: 45 | ID: 16

Departemen: Operations
Top 5 Karyawan:
Employee_82 | Gaji: $11592 | Usia: 30 | ID: 82
Employee_86 | Gaji: $11547 | Usia: 25 | ID: 86
Employee_74 | Gaji: $11460 | Usia: 23 | ID: 74
Employee_44 | Gaji: $11421 | Usia: 32 | ID: 44
Employee_75 | Gaji: $11175 | Usia: 38 | ID: 75

Departemen: Marketing
Top 5 Karyawan:
Employee_71 | Gaji: $8867 | Usia: 47 | ID: 71
Employee_3 | Gaji: $7808 | U

In [34]:
# Tugas 2: Menghapus karyawan yang berusia di bawah 25 tahun

# Menampilkan jumlah data sebelum penghapusan
print("Jumlah data sebelum dihapus:", collection.count_documents({}))

# Menghapus semua karyawan yang berusia di bawah 30 tahun
delete_result = collection.delete_many({'age': {'$lt': 30}})

# Menampilkan jumlah data yang dihapus
print(f"Jumlah data karyawan yang berusia di bawah 30 tahun yang dihapus: {delete_result.deleted_count}")

# Menampilkan jumlah data setelah penghapusan
print("Jumlah data setelah dihapus:", collection.count_documents({}))

# Menampilkan semua data karyawan yang tersisa untuk verifikasi
print("\nSemua Data Karyawan yang Tersisa Setelah Penghapusan:")
for employee in collection.find():
    print(employee)

Jumlah data sebelum dihapus: 84
Jumlah data karyawan yang berusia di bawah 30 tahun yang dihapus: 33
Jumlah data setelah dihapus: 51

Semua Data Karyawan yang Tersisa Setelah Penghapusan:
{'_id': ObjectId('690439e58d6b2495677813c2'), 'employee_id': 1, 'name': 'Employee_1', 'department': 'Sales', 'age': 34, 'salary': 5980}
{'_id': ObjectId('690439e58d6b2495677813c5'), 'employee_id': 4, 'name': 'Employee_4', 'department': 'IT', 'age': 35, 'salary': 8739.5}
{'_id': ObjectId('690439e58d6b2495677813c7'), 'employee_id': 6, 'name': 'Employee_6', 'department': 'Sales', 'age': 44, 'salary': 3449}
{'_id': ObjectId('690439e58d6b2495677813c9'), 'employee_id': 8, 'name': 'Employee_8', 'department': 'IT', 'age': 30, 'salary': 10913.1}
{'_id': ObjectId('690439e58d6b2495677813ce'), 'employee_id': 13, 'name': 'Employee_13', 'department': 'Operations', 'age': 44, 'salary': 6969}
{'_id': ObjectId('690439e58d6b2495677813cf'), 'employee_id': 14, 'name': 'Employee_14', 'department': 'IT', 'age': 47, 'salary

In [35]:
# Tugas 3
# Laporan ringkas: total gaji karyawan di setiap departemen dan rata-rata umur karyawan.

# Aggregation pipeline untuk menghitung total gaji dan rata-rata umur per departemen
pipeline = [
    {
        '$group': {
            '_id': '$department',
            'total_salary': {'$sum': '$salary'},
            'average_age': {'$avg': '$age'}
        }
    },
    {
        '$sort': {'_id': 1}  # Mengurutkan hasil berdasarkan nama departemen
    }
]

# Menampilkan hasil laporan
print("📋 Laporan Ringkas: Total Gaji dan Rata-rata Umur per Departemen\n")
for result in collection.aggregate(pipeline):
    print(f"Departemen: {result['_id']}")
    print(f"  Total Gaji     : {result['total_salary']}")
    print(f"  Rata-rata Umur : {result['average_age']:.2f} tahun\n")

📋 Laporan Ringkas: Total Gaji dan Rata-rata Umur per Departemen

Departemen: Finance
  Total Gaji     : 92470
  Rata-rata Umur : 40.42 tahun

Departemen: HR
  Total Gaji     : 52713
  Rata-rata Umur : 36.71 tahun

Departemen: IT
  Total Gaji     : 92687.1
  Rata-rata Umur : 39.30 tahun

Departemen: Marketing
  Total Gaji     : 30628
  Rata-rata Umur : 41.40 tahun

Departemen: Operations
  Total Gaji     : 66690
  Rata-rata Umur : 38.38 tahun

Departemen: Sales
  Total Gaji     : 57316
  Rata-rata Umur : 39.33 tahun



In [37]:
# Tugas Tambahan
# Membuat koleksi baru 'products'
products_collection = db["products"]

# Daftar kategori dan nama produk
categories = ["Elektronik", "Pakaian", "Makanan", "Peralatan Rumah", "Aksesoris", "Olahraga", "Kesehatan", "Mainan"]
product_names = [
    "Smartphone", "Laptop", "Kamera", "Headphone", "Keyboard", "Mouse", "Monitor", "TV",
    "Sepatu", "Jaket", "Kaos", "Celana", "Kemeja", "Topi", "Tas", "Jam Tangan",
    "Snack", "Minuman", "Roti", "Susu", "Coklat", "Kopi", "Mie Instan", "Saus",
    "Piring", "Sendok", "Kompor", "Vacuum Cleaner", "Blender", "Kulkas",
    "Cincin", "Kalung", "Gelang", "Anting", "Kacamata",
    "Bola", "Raket", "Treadmill", "Matras", "Sepeda",
    "Masker", "Vitamin", "Obat", "Suplemen", "Timbangan",
    "Puzzle", "Robot", "Boneka", "Lego", "Kartu Game"
]

# Membuat 100 data produk random
product_data = []
for i in range(100):
    product = {
        "product_id": i + 1,
        "name": random.choice(product_names) + f" {random.randint(100,999)}",
        "category": random.choice(categories),
        "price": random.randint(10000, 2000000),
        "stock": random.randint(0, 500),
        "rating": round(random.uniform(1.0, 5.0), 1)
    }
    product_data.append(product)

# Menambahkan data ke koleksi
products_collection.insert_many(product_data)

print("✅ Sebanyak 100 data produk berhasil dimasukkan ke koleksi 'products'!")

# Menampilkan data produk yang dibuat
print("\n📋 Data Produk yang Telah Dimasukkan:")
for product in products_collection.find():
    print(product)

✅ Sebanyak 100 data produk berhasil dimasukkan ke koleksi 'products'!

📋 Data Produk yang Telah Dimasukkan:
{'_id': ObjectId('69043c178d6b24956778148a'), 'product_id': 1, 'name': 'Minuman 767', 'category': 'Kesehatan', 'price': 700105, 'stock': 316, 'rating': 4.3}
{'_id': ObjectId('69043c178d6b24956778148b'), 'product_id': 2, 'name': 'Laptop 502', 'category': 'Makanan', 'price': 1357529, 'stock': 424, 'rating': 4.6}
{'_id': ObjectId('69043c178d6b24956778148c'), 'product_id': 3, 'name': 'Kompor 754', 'category': 'Elektronik', 'price': 1828699, 'stock': 117, 'rating': 3.9}
{'_id': ObjectId('69043c178d6b24956778148d'), 'product_id': 4, 'name': 'Kacamata 275', 'category': 'Pakaian', 'price': 499155, 'stock': 489, 'rating': 5.0}
{'_id': ObjectId('69043c178d6b24956778148e'), 'product_id': 5, 'name': 'Snack 695', 'category': 'Kesehatan', 'price': 501605, 'stock': 332, 'rating': 2.2}
{'_id': ObjectId('69043c178d6b24956778148f'), 'product_id': 6, 'name': 'Kemeja 882', 'category': 'Makanan', 'pr

In [40]:
# Latihan 4.2: Lakukan query untuk menemukan produk dengan harga di atas rata-rata.

# Hitung harga rata-rata semua produk
average_price_pipeline = [
    {'$group': {'_id': None, 'average_price': {'$avg': '$price'}}}
]
average_price_result = list(products_collection.aggregate(average_price_pipeline))
average_price = average_price_result[0]['average_price']

print(f"💵 Harga rata-rata semua produk: {average_price:.2f}\n")

# Query produk dengan harga di atas rata-rata
above_average_products = products_collection.find({'price': {'$gt': average_price}})

print("📦 Produk dengan Harga di Atas Rata-rata;")
for product in above_average_products:
    print(f"ID Produk   : {product['product_id']}")
    print(f"Nama Produk : {product['name']}")
    print(f"Kategori    : {product['category']}")
    print(f"Harga       : {product['price']}")
    print(f"Stok        : {product['stock']}")
    print(f"Rating      : {product['rating']}\n")

💵 Harga rata-rata semua produk: 1027616.96

📦 Produk dengan Harga di Atas Rata-rata;
ID Produk   : 2
Nama Produk : Laptop 502
Kategori    : Makanan
Harga       : 1357529
Stok        : 424
Rating      : 4.6

ID Produk   : 3
Nama Produk : Kompor 754
Kategori    : Elektronik
Harga       : 1828699
Stok        : 117
Rating      : 3.9

ID Produk   : 6
Nama Produk : Kemeja 882
Kategori    : Makanan
Harga       : 1210362
Stok        : 260
Rating      : 1.4

ID Produk   : 7
Nama Produk : Piring 243
Kategori    : Olahraga
Harga       : 1837856
Stok        : 454
Rating      : 3.3

ID Produk   : 8
Nama Produk : Tas 618
Kategori    : Olahraga
Harga       : 1642926
Stok        : 499
Rating      : 2.0

ID Produk   : 9
Nama Produk : Suplemen 864
Kategori    : Kesehatan
Harga       : 1269090
Stok        : 312
Rating      : 1.7

ID Produk   : 10
Nama Produk : Treadmill 788
Kategori    : Pakaian
Harga       : 1671809
Stok        : 349
Rating      : 3.9

ID Produk   : 11
Nama Produk : TV 987
Kategori    :

In [44]:
# Latihan 4.3: Buatlah aggregation pipeline untuk menghitung total produk dalam setiap kategori.
# Aggregation pipeline untuk menghitung total produk per kategori
pipeline = [
    {
        '$group': {
            '_id': '$category',
            'total_products': {'$sum': 1}
        }
    },
    {
        '$sort': {'total_products': -1}  # urutkan dari kategori dengan jumlah produk terbanyak
    }
]

# Menjalankan pipeline dan menampilkan hasil
print("Total Produk per Kategori: \n")
for result in products_collection.aggregate(pipeline):
    print(f"Kategori       : {result['_id']}")
    print(f"Total Produk  : {result['total_products']}\n")

Total Produk per Kategori: 

Kategori       : Elektronik
Total Produk  : 16

Kategori       : Aksesoris
Total Produk  : 14

Kategori       : Olahraga
Total Produk  : 14

Kategori       : Kesehatan
Total Produk  : 13

Kategori       : Mainan
Total Produk  : 12

Kategori       : Makanan
Total Produk  : 11

Kategori       : Peralatan Rumah
Total Produk  : 10

Kategori       : Pakaian
Total Produk  : 10

